# Bounded Box Gridded Data
This notebook demonstrates the extraction of mapped data for a bounding box. These kind of requests return GeoTIFF type files. We will specify a bounding box and download Planetary Variable gridded-data.

Reference information can be found at [Data access - Requesting gridded data](https://docs.vandersat.com/VanderSat_API_UserGuide.html#asynchronous-using-the-gridded-data-endpoint)

Other API-endpoint operations can be found at [REST API - Planetary Variable products](https://maps.vandersat.com/api/v2/)

In [ ]:
import os 
import requests

## Authenticating

In [ ]:
# Username and API Key stored as an environment variable
AUTH = (os.getenv("VDS_USER"), os.getenv("VDS_PASS"))

In [ ]:
BASE_URL = "https://maps.vandersat.com/api/v2"

## Requests example
First, we will check if our authentication is working. We want a response code of `200` from this API call.

In [ ]:
# Authenticate
session = requests.Session()
session.auth = AUTH

# Request
response = session.get(BASE_URL)
response

## Product operation examples
We can request a list of the account's available PVs and see their information.

In [ ]:
PRODUCT_URL = os.path.join(BASE_URL, "products")

In [ ]:
response = session.get(PRODUCT_URL)
product_list = response.json()["products"]
product_list

We can request a product's gridded data for a bounding box; we only need to define which `product_name`, and parameters `lon_min` & `lon_max`& `lat_min` & `lat_max` & `start_date` & `end_date`. Make sure the bounding box is inside your account's area-allowed.

In [ ]:
product_name = product_list[0]["api_name"]  # e.g. 'TEFF-AMSR2-DESC_V4.0_1000'
parameters = {  # e.g. Netherlands
    "lon_min": "4.909772872924805",
    "lon_max": "4.922990798950195",
    "lat_min": "51.989637331792146",
    "lat_max": "51.99328417001946",
    "start_date": "2022-07-01",
    "end_date": "2022-07-03"
}

In [ ]:
GRIDDED_URL = os.path.join(PRODUCT_URL, product_name, "gridded-data")
response = session.get(GRIDDED_URL, params=parameters)
response.json()

We've sent a download request, which takes our system some time to process. The request's response gives us an URL to check the `processing_status`. Next, when processing is ready, `percentage` returns 100, and `data` returns a list of downloadable file links.

In [ ]:
STATUS_URL = os.path.join("https://maps.vandersat.com", response.json()["url"])

status = session.get(STATUS_URL).json()
status

Let's request the `processing_status` and download the files when ready.

In [ ]:
import time

# Specify output folder
output_folder = "/home/guyon/Downloads"

# Check status
while status["percentage"] < 100:
    print(f"Percentage:", f"{status['percentage']}%")
    time.sleep(3)
    status = session.get(STATUS_URL).json()
print("Processing status 100%")

# Download files
files = status["data"]
for i, file in enumerate(files):
    print("Downloading:", f"{i+1}/{len(files)}")
    file_uri = f"https://maps.vandersat.com/{file}/download"
    r = session.get(file_uri)
    
    file_name = os.path.basename(file)
    file_path = os.path.join(output_folder, file_name)
    with open(file_path, 'wb') as f:
        for chunk in r.iter_content(1024):
            f.write(chunk)
print("Finished downloading")
    